In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import Counter
from tqdm import tqdm

import joblib

import numpy as np
import pandas as pd

from transformers import AutoTokenizer

from pygments import lex
from pygments.lexers import get_lexer_by_name, guess_lexer, find_lexer_class_by_name, get_all_lexers
from pygments.util import ClassNotFound
from pygments.token import Token

from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from magika import Magika


def batchify_list(data_list, batch_size):
    """Yield successive n-sized chunks from data_list."""
    for i in range(0, len(data_list), batch_size):
        yield data_list[i:i + batch_size]

def flatten_list(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]

def list_supported_languages():
    # get_all_lexers() returns an iterator of tuples: 
    # (Long Name, (Aliases), (File Patterns), (Mime Types))
    lexers = list(get_all_lexers())
    
    # Sort them alphabetically by long name
    lexers.sort(key=lambda x: x[0])

    print(f"{'Language Name':<30} | {'Aliases (Use these IDs)'}")
    print("-" * 70)
    
    for name, aliases, _, _ in lexers:
        # Join aliases with commas for display
        alias_str = ", ".join(aliases)
        print(f"{name:<30} | {alias_str}")

# list_supported_languages()

def tokenize_code(code_string, language=None):
    lexer = None
    detected_lang = "Unknown"

    # --- STRATEGY 1: Explicit Language (Hard Override) ---
    if language:
        try:
            lexer = get_lexer_by_name(language)
            detected_lang = lexer.name
        except ClassNotFound:
            print(f"Warning: Lexer for '{language}' not found. Falling back to detection.")

    # --- Tokenization ---
    raw_tokens = lex(code_string, lexer)
    structured_tokens = []
    
    for token_type, value in raw_tokens:
        if not value: continue
        
        token_data = {
            "text": value,
            "type": str(token_type),
            "base_type": str(token_type).split('.')[1] if '.' in str(token_type) else "Text"
        }
        structured_tokens.append(token_data)

    return {
        "language": detected_lang,
        "tokens": structured_tokens
    }

semeval2pygment_ids = {
    "C++": "c++",
    "Python": "python",
    "Java": "java",
    "Go": "go",
    "PHP": "php",
    "C#": "c#",
    "C": "c",
    "JS": "javascript",
}
pygment2semeval_ids = {v: k for k, v in semeval2pygment_ids.items()}

/Users/rendvvv/miniconda3/envs/lingu/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load data
N_SAMPLE = 10_000

D_tr = pd.read_parquet("./data/task_a_training_set_1.parquet").sample(n=N_SAMPLE)
D_ev = pd.read_parquet("./data/task_a_validation_set.parquet").sample(n=N_SAMPLE)
D_te = pd.read_parquet("./data/test.parquet")

print(D_tr.shape, D_ev.shape, D_te.shape)
D_tr.head(2)

(10000, 4) (10000, 4) (1000, 2)


,code,generator,label,language
77846,"python\na,b,c = map(int, input().split())\n\nm...",meta-llama/Llama-3.1-8B-Instruct,1,Python
273247,import math\nimport sys\ninput = sys.stdin.rea...,human,0,Python


In [4]:
# Tokenize codes
D_tr["token_obj"] = [tokenize_code(code, language=semeval2pygment_ids[lang]) for code, lang in tqdm(zip(D_tr["code"], D_tr["language"]), total=len(D_tr))]
D_tr["tokens"] = [obj["tokens"] for obj in D_tr["token_obj"]]

D_ev["token_obj"] = [tokenize_code(code, language=semeval2pygment_ids[lang]) for code, lang in tqdm(zip(D_ev["code"], D_ev["language"]), total=len(D_ev))]
D_ev["tokens"] = [obj["tokens"] for obj in D_ev["token_obj"]]

100%|██████████| 10000/10000 [00:11<00:00, 850.83it/s]


In [5]:
# Prettify features
D_tr["base"] = [[token["base_type"] for token in tokens] for tokens in D_tr["tokens"]]
D_ev["base"] = [[token["base_type"] for token in tokens] for tokens in D_ev["tokens"]]

D_tr["text"] = [[token["text"] for token in tokens] for tokens in D_tr["tokens"]]
D_ev["text"] = [[token["text"] for token in tokens] for tokens in D_ev["tokens"]]

D_tr["text_base"] = [[(token["text"], token["base_type"]) for token in tokens] for tokens in D_tr["tokens"]]
D_ev["text_base"] = [[(token["text"], token["base_type"]) for token in tokens] for tokens in D_ev["tokens"]]

In [6]:
# Initiate language predictor
from plid import PlidWithMagika

lang_predictor = PlidWithMagika()

In [7]:
# Evaluate language predictor
# D_ev["language_pred"] = [lang_predictor.identify(t, n_segments=3, overlap_ratio=0.5) for t in tqdm(D_ev["code"])]
# print(classification_report(D_ev["language"], D_ev["language_pred"]))

In [8]:
# Define vocabularies
VOCAB_TEXT = list(set(flatten_list(D_tr["text"].to_list())))
VOCAB_BASE = list(set(flatten_list(D_tr["base"].to_list())))
VOCAB_TEXT_BASE = list(set(flatten_list(D_tr["text_base"].to_list())))

In [9]:
VOCAB_BASE

['Literal',
 'Name',
 'Text',
 'Keyword',
 'Comment',
 'Operator',
 'Error',
 'Punctuation']

In [10]:
# Split data to human and machine
D_human_tr = D_tr[D_tr["generator"] == "human"]
D_human_ev = D_ev[D_ev["generator"] == "human"]

D_machine_tr = D_tr[D_tr["generator"] != "human"]
D_machine_ev = D_ev[D_ev["generator"] != "human"]

In [11]:
BASE_FEATURE_LIST = ["Text", "Comment"]

In [12]:
from sentence_transformers import SentenceTransformer

text_embedder = SentenceTransformer("google/embeddinggemma-300m", device="mps")

In [13]:
# Feature from comments
comments_tr_human = flatten_list([[t for t, b in tbs if b == "Comment"] for tbs in tqdm(D_human_tr["text_base"])])
comments_tr_human = list(set(comments_tr_human))
comments_tr_human = [x for x in comments_tr_human if x != ""]

comments_tr_machine = flatten_list([[t for t, b in tbs if b == "Comment"] for tbs in tqdm(D_machine_tr["text_base"])])
comments_tr_machine = list(set(comments_tr_machine))
comments_tr_machine = [x for x in comments_tr_machine if x != ""]

print(len(comments_tr_human), len(comments_tr_machine))

100%|██████████| 5170/5170 [00:00<00:00, 71513.55it/s]

2251 10608


In [ ]:
BATCH_SIZE = 32

embs = []
for batch in tqdm(batchify_list(comments_tr_human, BATCH_SIZE), total=len(comments_tr_human) // BATCH_SIZE):
    embs.extend(text_embedder.encode(batch).tolist())
comments_tr_human_embs = np.array(embs)

embs = []
for batch in tqdm(batchify_list(comments_tr_machine, BATCH_SIZE), total=len(comments_tr_machine) // BATCH_SIZE):
    embs.extend(text_embedder.encode(batch).tolist())
comments_tr_machine_embs = np.array(embs)

 60%|██████    | 6/10 [00:27<00:18,  4.54s/it]


RuntimeError: MPS backend out of memory (MPS allocated: 5.18 GiB, other allocations: 12.95 GiB, max allowed: 18.13 GiB). Tried to allocate 13.59 MiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

pca = PCA(n_components=2)
pca.fit(comments_tr_human_embs + comments_tr_machine_embs)

comments_tr_human_embs_pcs = pca.transform(comments_tr_human_embs)
comments_tr_machine_embs_pcs = pca.transform(comments_tr_machine_embs)

plt.scatter(comments_tr_human_embs_pcs[:, 0], comments_tr_human_embs_pcs[:, 1])
plt.scatter(comments_tr_machine_embs_pcs[:, 0], comments_tr_machine_embs_pcs[:, 1])
plt.show()




ValueError: operands could not be broadcast together with shapes (2321,2) (10628,2) 

In [53]:
comments_tr_human_embs + comments_tr_machine_embs

ValueError: operands could not be broadcast together with shapes (2321,2) (10628,2) 

In [34]:
flatten_list([[t for t, b in tbs if b == "Punctuation"] for tbs in tqdm(D_machine_tr["text_base"])])

100%|██████████| 5244/5244 [00:00<00:00, 69681.39it/s]


['(',
 ',',
 ',',
 ')',
 ':',
 '[',
 '(',
 ',',
 ')',
 ',',
 '(',
 ',',
 ')',
 ',',
 '(',
 ',',
 ')',
 ']',
 '(',
 '(',
 ')',
 ')',
 '(',
 '(',
 ',',
 ',',
 ')',
 ')',
 '(',
 ',',
 ')',
 ':',
 ':',
 '(',
 '(',
 ',',
 ')',
 ')',
 ',',
 ':',
 ',',
 ',',
 '(',
 ',',
 ')',
 ':',
 '[',
 ']',
 '[',
 ']',
 '(',
 ',',
 ')',
 '(',
 ',',
 ')',
 '(',
 '(',
 ',',
 ')',
 ')',
 ':',
 '(',
 ',',
 '(',
 ',',
 ')',
 ')',
 '(',
 ')',
 ':',
 ':',
 ',',
 '(',
 ',',
 ')',
 '(',
 ')',
 '(',
 ')',
 ':',
 ':',
 '(',
 ')',
 '(',
 ')',
 '(',
 '(',
 ')',
 ')',
 '(',
 ')',
 ':',
 '(',
 ')',
 '(',
 ')',
 ':',
 ',',
 '[',
 ':',
 ']',
 '(',
 ')',
 ',',
 '(',
 ')',
 ',',
 '(',
 ')',
 '(',
 ')',
 '(',
 '(',
 ')',
 ')',
 ':',
 ':',
 '(',
 ')',
 ':',
 ':',
 '(',
 ')',
 '(',
 ')',
 '(',
 '(',
 ')',
 '(',
 ')',
 ')',
 '(',
 ')',
 ':',
 ',',
 '(',
 ',',
 '(',
 ')',
 '(',
 ')',
 ')',
 '{',
 '}',
 '(',
 ')',
 ':',
 ',',
 '(',
 ')',
 '(',
 ')',
 '[',
 ']',
 '[',
 ']',
 '(',
 ')',
 '(',
 ')',
 ':',
 '(',
 ',',
 '[',
 ']',
 ','

In [35]:
# Build features
len(Counter(flatten_list(D_tr["text_base"].to_list())))

69742

In [69]:
def identity_fn(x):
    return x

plid_vectorizer = TfidfVectorizer(
    vocabulary=VOCAB_FOR_PLID,
    analyzer=identity_fn,
    lowercase=False,
    norm=None,
    use_idf=True,
    smooth_idf=True,
).fit(D_tr["text_base"].to_list())

In [70]:
X_tr = plid_vectorizer.transform(D_tr["text_base"].to_list())
X_ev = plid_vectorizer.transform(D_ev["text_base"].to_list())

In [71]:
plid = LogisticRegression(
    penalty="l2",
    C=1.0,
    solver="lbfgs",
    max_iter=1000,
    random_state=888,
).fit(X_tr, D_tr["language"])

In [72]:
print(classification_report(D_ev["language"], plid.predict(X_ev)))

              precision    recall  f1-score   support

         C++       0.98      0.97      0.98       459
        Java       0.97      0.97      0.97       413
      Python       1.00      1.00      1.00      9128

    accuracy                           1.00     10000
   macro avg       0.99      0.98      0.98     10000
weighted avg       1.00      1.00      1.00     10000



In [ ]:
for lang in D_tr["language"].unique():
    D_tr_sub = D_tr[D_tr["language"] == lang]

    

Python
Java
C++


In [57]:
len(VOCAB_FOR_PLID)

132

In [38]:
D_tr

,code,generator,label,language,token_obj,tokens,base_type,text,base,text_base
361965,t = int(input())\nfor _ in range(t):\n\tk = [l...,human,0,Python,"{'language': 'Python', 'tokens': [{'text': 't'...","[{'text': 't', 'type': 'Token.Name', 'base_typ...","[Name, Text, Operator, Text, Name, Punctuation...","[t, , =, , int, (, input, (, ), ), \n, for, ...","[Name, Text, Operator, Text, Name, Punctuation...","[t<->Name, <->Text, =<->Operator, <->Text, i..."
179837,import static java.lang.Math.max;\n\nimport st...,human,0,Java,"{'language': 'Java', 'tokens': [{'text': 'impo...","[{'text': 'import static', 'type': 'Token.Keyw...","[Keyword, Text, Name, Punctuation, Text, Text,...","[import static, , java.lang.Math.max, ;, \n, ...","[Keyword, Text, Name, Punctuation, Text, Text,...","[import static<->Keyword, <->Text, java.lang...."
472639,"(n, *a) = map(int, open(0).read().split())\nm ...",human,0,Python,"{'language': 'Python', 'tokens': [{'text': '('...","[{'text': '(', 'type': 'Token.Punctuation', 'b...","[Punctuation, Name, Punctuation, Text, Operato...","[(, n, ,, , *, a, ), , =, , map, (, int, ,,...","[Punctuation, Name, Punctuation, Text, Operato...","[(<->Punctuation, n<->Name, ,<->Punctuation, ..."
97298,from collections import defaultdict\n\ndef are...,human,0,Python,"{'language': 'Python', 'tokens': [{'text': 'fr...","[{'text': 'from', 'type': 'Token.Keyword.Names...","[Keyword, Text, Name, Text, Keyword, Text, Nam...","[from, , collections, , import, , defaultdi...","[Keyword, Text, Name, Text, Keyword, Text, Nam...","[from<->Keyword, <->Text, collections<->Name,..."
276933,"n = int(input())\n(v, a) = ([0] * n, [0] * n)\...",human,0,Python,"{'language': 'Python', 'tokens': [{'text': 'n'...","[{'text': 'n', 'type': 'Token.Name', 'base_typ...","[Name, Text, Operator, Text, Name, Punctuation...","[n, , =, , int, (, input, (, ), ), \n, (, v,...","[Name, Text, Operator, Text, Name, Punctuation...","[n<->Name, <->Text, =<->Operator, <->Text, i..."
...,...,...,...,...,...,...,...,...,...,...
448969,bisect.bisect_left can be helpful in finding v...,microsoft/phi-2,1,Python,"{'language': 'Python', 'tokens': [{'text': 'bi...","[{'text': 'bisect', 'type': 'Token.Name', 'bas...","[Name, Operator, Name, Text, Name, Text, Name,...","[bisect, ., bisect_left, , can, , be, , hel...","[Name, Operator, Name, Text, Name, Text, Name,...","[bisect<->Name, .<->Operator, bisect_left<->Na..."
416400,", don't print anything.\ndef beautiful_string_...",Qwen/Qwen2.5-Coder-1.5B,1,Python,"{'language': 'Python', 'tokens': [{'text': ','...","[{'text': ',', 'type': 'Token.Punctuation', 'b...","[Punctuation, Text, Name, Literal, Literal, Te...","[,, , don, ', t print anything., \n, def, , ...","[Punctuation, Text, Name, Literal, Literal, Te...","[,<->Punctuation, <->Text, don<->Name, '<->Li..."
61662,const int MAXN = 300005;\n\nnamespace Trie {\n...,human,0,C++,"{'language': 'C++', 'tokens': [{'text': 'const...","[{'text': 'const', 'type': 'Token.Keyword', 'b...","[Keyword, Text, Keyword, Text, Name, Text, Ope...","[const, , int, , MAXN, , =, , 300005, ;, \...","[Keyword, Text, Keyword, Text, Name, Text, Ope...","[const<->Keyword, <->Text, int<->Keyword, <-..."
337637,python\n\ndef min_rectangle_cover(points):\n ...,01-ai/Yi-Coder-9B-Chat,1,Python,"{'language': 'Python', 'tokens': [{'text': 'py...","[{'text': 'python', 'type': 'Token.Name', 'bas...","[Name, Text, Text, Keyword, Text, Name, Punctu...","[python, \n, \n, def, , min_rectangle_cover, ...","[Name, Text, Text, Keyword, Text, Name, Punctu...","[python<->Name, \n<->Text, \n<->Text, def<->Ke..."


In [36]:
flatten_list(D_tr["text_base"].to_list())

['t<->Name',
 ' <->Text',
 '=<->Operator',
 ' <->Text',
 'int<->Name',
 '(<->Punctuation',
 'input<->Name',
 '(<->Punctuation',
 ')<->Punctuation',
 ')<->Punctuation',
 '\n<->Text',
 'for<->Keyword',
 ' <->Text',
 '_<->Name',
 ' <->Text',
 'in<->Operator',
 ' <->Text',
 'range<->Name',
 '(<->Punctuation',
 't<->Name',
 ')<->Punctuation',
 ':<->Punctuation',
 '\n<->Text',
 '\t<->Text',
 'k<->Name',
 ' <->Text',
 '=<->Operator',
 ' <->Text',
 '[<->Punctuation',
 'len<->Name',
 '(<->Punctuation',
 'x<->Name',
 ')<->Punctuation',
 ' <->Text',
 'for<->Keyword',
 ' <->Text',
 'x<->Name',
 ' <->Text',
 'in<->Operator',
 ' <->Text',
 'input<->Name',
 '(<->Punctuation',
 ')<->Punctuation',
 '.<->Operator',
 'split<->Name',
 '(<->Punctuation',
 "'<->Literal",
 'R<->Literal',
 "'<->Literal",
 ')<->Punctuation',
 ']<->Punctuation',
 '\n<->Text',
 '\t<->Text',
 'print<->Name',
 '(<->Punctuation',
 'max<->Name',
 '(<->Punctuation',
 'k<->Name',
 ')<->Punctuation',
 ' <->Text',
 '+<->Operator',
 ' <-

In [ ]:
# Build pLID

In [32]:
D_tr

,code,generator,label,language,token_obj,tokens,base_type,text,base,text_base
361965,t = int(input())\nfor _ in range(t):\n\tk = [l...,human,0,Python,"{'language': 'Python', 'tokens': [{'text': 't'...","[{'text': 't', 'type': 'Token.Name', 'base_typ...","[Name, Text, Operator, Text, Name, Punctuation...","[t, , =, , int, (, input, (, ), ), \n, for, ...","[Name, Text, Operator, Text, Name, Punctuation...","[t<->Name, <->Text, =<->Operator, <->Text, i..."
179837,import static java.lang.Math.max;\n\nimport st...,human,0,Java,"{'language': 'Java', 'tokens': [{'text': 'impo...","[{'text': 'import static', 'type': 'Token.Keyw...","[Keyword, Text, Name, Punctuation, Text, Text,...","[import static, , java.lang.Math.max, ;, \n, ...","[Keyword, Text, Name, Punctuation, Text, Text,...","[import static<->Keyword, <->Text, java.lang...."
472639,"(n, *a) = map(int, open(0).read().split())\nm ...",human,0,Python,"{'language': 'Python', 'tokens': [{'text': '('...","[{'text': '(', 'type': 'Token.Punctuation', 'b...","[Punctuation, Name, Punctuation, Text, Operato...","[(, n, ,, , *, a, ), , =, , map, (, int, ,,...","[Punctuation, Name, Punctuation, Text, Operato...","[(<->Punctuation, n<->Name, ,<->Punctuation, ..."
97298,from collections import defaultdict\n\ndef are...,human,0,Python,"{'language': 'Python', 'tokens': [{'text': 'fr...","[{'text': 'from', 'type': 'Token.Keyword.Names...","[Keyword, Text, Name, Text, Keyword, Text, Nam...","[from, , collections, , import, , defaultdi...","[Keyword, Text, Name, Text, Keyword, Text, Nam...","[from<->Keyword, <->Text, collections<->Name,..."
276933,"n = int(input())\n(v, a) = ([0] * n, [0] * n)\...",human,0,Python,"{'language': 'Python', 'tokens': [{'text': 'n'...","[{'text': 'n', 'type': 'Token.Name', 'base_typ...","[Name, Text, Operator, Text, Name, Punctuation...","[n, , =, , int, (, input, (, ), ), \n, (, v,...","[Name, Text, Operator, Text, Name, Punctuation...","[n<->Name, <->Text, =<->Operator, <->Text, i..."
...,...,...,...,...,...,...,...,...,...,...
448969,bisect.bisect_left can be helpful in finding v...,microsoft/phi-2,1,Python,"{'language': 'Python', 'tokens': [{'text': 'bi...","[{'text': 'bisect', 'type': 'Token.Name', 'bas...","[Name, Operator, Name, Text, Name, Text, Name,...","[bisect, ., bisect_left, , can, , be, , hel...","[Name, Operator, Name, Text, Name, Text, Name,...","[bisect<->Name, .<->Operator, bisect_left<->Na..."
416400,", don't print anything.\ndef beautiful_string_...",Qwen/Qwen2.5-Coder-1.5B,1,Python,"{'language': 'Python', 'tokens': [{'text': ','...","[{'text': ',', 'type': 'Token.Punctuation', 'b...","[Punctuation, Text, Name, Literal, Literal, Te...","[,, , don, ', t print anything., \n, def, , ...","[Punctuation, Text, Name, Literal, Literal, Te...","[,<->Punctuation, <->Text, don<->Name, '<->Li..."
61662,const int MAXN = 300005;\n\nnamespace Trie {\n...,human,0,C++,"{'language': 'C++', 'tokens': [{'text': 'const...","[{'text': 'const', 'type': 'Token.Keyword', 'b...","[Keyword, Text, Keyword, Text, Name, Text, Ope...","[const, , int, , MAXN, , =, , 300005, ;, \...","[Keyword, Text, Keyword, Text, Name, Text, Ope...","[const<->Keyword, <->Text, int<->Keyword, <-..."
337637,python\n\ndef min_rectangle_cover(points):\n ...,01-ai/Yi-Coder-9B-Chat,1,Python,"{'language': 'Python', 'tokens': [{'text': 'py...","[{'text': 'python', 'type': 'Token.Name', 'bas...","[Name, Text, Text, Keyword, Text, Name, Punctu...","[python, \n, \n, def, , min_rectangle_cover, ...","[Name, Text, Text, Keyword, Text, Name, Punctu...","[python<->Name, \n<->Text, \n<->Text, def<->Ke..."


In [25]:
Counter(flatten_list(D_tr["base_type"].to_list()))

Counter({'Text': 1285889,
         'Name': 741223,
         'Punctuation': 716371,
         'Operator': 393538,
         'Literal': 268842,
         'Keyword': 139774,
         'Comment': 24032,
         'Error': 9882})

In [26]:
Counter(flatten_list(D_ev["base_type"].to_list()))

Counter({'Text': 1276265,
         'Name': 733407,
         'Punctuation': 715175,
         'Operator': 386844,
         'Literal': 264753,
         'Keyword': 137690,
         'Comment': 22545,
         'Error': 9876})

In [ ]:
# Predict language
D_tr["language_pred"] = [
    lang_detector.identify_bytes(f"{code}".encode('utf-8')).output.label
    for code in tqdm(D_tr["code"], total=len(D_tr))
]

D_ev["language_pred"] = [
    lang_detector.identify_bytes(f"{code}".encode('utf-8')).output.label
    for code in tqdm(D_ev["code"], total=len(D_ev))
]

D_te["language_pred"] = [
    lang_detector.identify_bytes(f"{code}".encode('utf-8')).output.label
    for code in tqdm(D_te["code"], total=len(D_te))
]

100%|██████████| 1000/1000 [00:03<00:00, 307.83it/s]


In [ ]:
semeval2pygment_ids = {
    "C++": "cpp",
    "Python": "python",
    "Java": "java",
    "Go": "go",
    "PHP": "php",
    "C#": "c#",
    "C": "c",
    "JS": "javascript",
}

In [23]:
# Tokenize code
D_tr["token_obj"] = [tokenize_code(code, language=lang) for code, lang in tqdm(zip(D_tr["code"], D_tr["language"]), total=len(D_tr))]
D_tr["tokens"] = [obj["tokens"] for obj in D_tr["token_obj"]]

D_ev["token_obj"] = [tokenize_code(code, possible_languages=semeval2pygment_ids.values()) for code, lang in tqdm(zip(D_ev["code"], D_ev["language"]), total=len(D_ev))]
D_ev["tokens"] = [obj["tokens"] for obj in D_ev["token_obj"]]

D_te["token_obj"] = [tokenize_code(code, possible_languages=semeval2pygment_ids.values()) for code, lang in tqdm(zip(D_te["code"], D_te["language"]), total=len(D_te))]
D_te["tokens"] = [obj["tokens"] for obj in D_te["token_obj"]]

100%|██████████| 1000/1000 [00:01<00:00, 849.96it/s]


In [29]:
D_ev["language"]

24602    python
94229      java
64802    python
7583     python
16997      java
          ...  
63690    python
59449    python
19887    python
81512    python
80226    python
Name: language, Length: 10000, dtype: object

In [32]:
D_ev["token_obj"].apply(lambda x: semeval2pygment_ids[x["language"]])

24602       c++
94229       c++
64802       c++
7583        c++
16997    python
          ...  
63690    python
59449       c++
19887       c++
81512    python
80226       c++
Name: token_obj, Length: 10000, dtype: object

In [34]:
!pip install magika

  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached flatbuffers-25.9.23-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 15.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 24.5 MB/s  0:00:00 eta 0:00:01
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)
Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)
Using cached flatbuffers-25.9.23-py2.py3-none-any.whl (30 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [magika]2m4/6 [onnxruntime]


In [36]:
m = Magika()
res = m.identify_bytes(b'function log(msg) {console.log(msg);}')
print(res.output.label)

javascript


In [33]:
print(classification_report(D_ev["language"], D_ev["token_obj"].apply(lambda x: semeval2pygment_ids[x["language"]])))

              precision    recall  f1-score   support

           c       0.00      0.00      0.00         0
         c++       0.06      0.97      0.11       451
        java       0.00      0.00      0.00       384
         php       0.00      0.00      0.00         0
      python       0.93      0.26      0.40      9165

    accuracy                           0.28     10000
   macro avg       0.20      0.24      0.10     10000
weighted avg       0.86      0.28      0.37     10000



/Users/rendvvv/miniconda3/envs/lingu/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/rendvvv/miniconda3/envs/lingu/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/rendvvv/miniconda3/envs/lingu/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

In [18]:
sample["tokens"]

[{'text': 'def', 'type': 'Token.Keyword', 'base_type': 'Keyword'},
 {'text': ' ', 'type': 'Token.Text.Whitespace', 'base_type': 'Text'},
 {'text': 'range_search', 'type': 'Token.Name.Function', 'base_type': 'Name'},
 {'text': '(', 'type': 'Token.Punctuation', 'base_type': 'Punctuation'},
 {'text': 'points', 'type': 'Token.Name', 'base_type': 'Name'},
 {'text': ',', 'type': 'Token.Punctuation', 'base_type': 'Punctuation'},
 {'text': ' ', 'type': 'Token.Text', 'base_type': 'Text'},
 {'text': 'queries', 'type': 'Token.Name', 'base_type': 'Name'},
 {'text': ')', 'type': 'Token.Punctuation', 'base_type': 'Punctuation'},
 {'text': ':', 'type': 'Token.Punctuation', 'base_type': 'Punctuation'},
 {'text': '\n', 'type': 'Token.Text.Whitespace', 'base_type': 'Text'},
 {'text': '    ', 'type': 'Token.Text', 'base_type': 'Text'},
 {'text': 'results', 'type': 'Token.Name', 'base_type': 'Name'},
 {'text': ' ', 'type': 'Token.Text', 'base_type': 'Text'},
 {'text': '=', 'type': 'Token.Operator', 'base_

In [8]:
# Your input code string
code_snippet = """class Solution:

\tdef maximumSum(self, arr: List[int]) -> int:
\t\tr = max(arr)
\t\tif r < 0:
\t\t\treturn r
\t\tl = len(arr)
\t\tf = [0] * l
\t\tb = [0] * l
\t\tcur = 0
\t\tfor i in range(l):
\t\t\tf[i] = cur
\t\t\tcur += arr[i]
\t\t\tcur = max(cur, 0)
\t\tcur = 0
\t\tfor i in range(l - 1, -1, -1):
\t\t\tb[i] = cur
\t\t\tcur += arr[i]
\t\t\tcur = max(cur, 0)
\t\treturn max((f[i] + b[i] for i in range(l)))
"""

def tokenize_code(code):
    # The lex() function returns a generator of (token_type, value) tuples
    tokens = list(lex(code, PythonLexer()))
    
    print(f"{'Token Type':<30} | {'Value (repr)':<20}")
    print("-" * 55)
    
    for token_type, value in tokens:
        # We use repr(value) to see hidden characters like \t and \n clearly
        print(f"{str(token_type):<30} | {repr(value):<20}")

tokenize_code(code_snippet)

Token Type                     | Value (repr)        
-------------------------------------------------------
Token.Keyword                  | 'class'             
Token.Text.Whitespace          | ' '                 
Token.Name.Class               | 'Solution'          
Token.Punctuation              | ':'                 
Token.Text.Whitespace          | '\n'                
Token.Text.Whitespace          | '\n'                
Token.Text                     | '\t'                
Token.Keyword                  | 'def'               
Token.Text.Whitespace          | ' '                 
Token.Name.Function            | 'maximumSum'        
Token.Punctuation              | '('                 
Token.Name.Builtin.Pseudo      | 'self'              
Token.Punctuation              | ','                 
Token.Text                     | ' '                 
Token.Name                     | 'arr'               
Token.Punctuation              | ':'                 
Token.Text                

In [54]:
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder")

In [58]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Coder-32B-Instruct")

(500000, 4) (100000, 4) (1000, 4)


,code,generator,label,language
0,"(a, b, c, d) = [int(x) for x in input().split(...",human,0,Python
1,valid version for the language; all others can...,Qwen/Qwen2.5-Coder-1.5B,1,Python


In [63]:
sample = D_tr.sample().iloc[0]

print(sample["code"])

class Solution:

	def maximumSum(self, arr: List[int]) -> int:
		r = max(arr)
		if r < 0:
			return r
		l = len(arr)
		f = [0] * l
		b = [0] * l
		cur = 0
		for i in range(l):
			f[i] = cur
			cur += arr[i]
			cur = max(cur, 0)
		cur = 0
		for i in range(l - 1, -1, -1):
			b[i] = cur
			cur += arr[i]
			cur = max(cur, 0)
		return max((f[i] + b[i] for i in range(l)))



In [75]:
from nltk import ngrams, word_tokenize

In [86]:
!pip install Pygments

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
sample["code"]

'class Solution:\n\n\tdef maximumSum(self, arr: List[int]) -> int:\n\t\tr = max(arr)\n\t\tif r < 0:\n\t\t\treturn r\n\t\tl = len(arr)\n\t\tf = [0] * l\n\t\tb = [0] * l\n\t\tcur = 0\n\t\tfor i in range(l):\n\t\t\tf[i] = cur\n\t\t\tcur += arr[i]\n\t\t\tcur = max(cur, 0)\n\t\tcur = 0\n\t\tfor i in range(l - 1, -1, -1):\n\t\t\tb[i] = cur\n\t\t\tcur += arr[i]\n\t\t\tcur = max(cur, 0)\n\t\treturn max((f[i] + b[i] for i in range(l)))\n'

In [84]:
def get_bow_ngrams_from_doc(text, n=1):
    tokens = [word for word in text.split(" ")]
    tokens = list(ngrams(tokens, n=n))
    tokens = [" ".join(token) for token in tokens]
    return tokens

In [85]:
get_bow_ngrams_from_doc(sample["code"])

['class',
 'Solution:\n\n\tdef',
 'maximumSum(self,',
 'arr:',
 'List[int])',
 '->',
 'int:\n\t\tr',
 '=',
 'max(arr)\n\t\tif',
 'r',
 '<',
 '0:\n\t\t\treturn',
 'r\n\t\tl',
 '=',
 'len(arr)\n\t\tf',
 '=',
 '[0]',
 '*',
 'l\n\t\tb',
 '=',
 '[0]',
 '*',
 'l\n\t\tcur',
 '=',
 '0\n\t\tfor',
 'i',
 'in',
 'range(l):\n\t\t\tf[i]',
 '=',
 'cur\n\t\t\tcur',
 '+=',
 'arr[i]\n\t\t\tcur',
 '=',
 'max(cur,',
 '0)\n\t\tcur',
 '=',
 '0\n\t\tfor',
 'i',
 'in',
 'range(l',
 '-',
 '1,',
 '-1,',
 '-1):\n\t\t\tb[i]',
 '=',
 'cur\n\t\t\tcur',
 '+=',
 'arr[i]\n\t\t\tcur',
 '=',
 'max(cur,',
 '0)\n\t\treturn',
 'max((f[i]',
 '+',
 'b[i]',
 'for',
 'i',
 'in',
 'range(l)))\n']

In [59]:
tokenizer.tokenize(sample["code"])

['def',
 'Ġnth',
 'Mag',
 'ical',
 'Number',
 '(N',
 ':',
 'Ġint',
 ',',
 'ĠA',
 ':',
 'Ġint',
 ',',
 'ĠB',
 ':',
 'Ġint',
 ')',
 'Ġ->',
 'Ġint',
 ':Ċ',
 'ĠĠĠ',
 'ĠMOD',
 'Ġ=',
 'Ġ',
 '1',
 '0',
 '**',
 '9',
 'Ġ+',
 'Ġ',
 '7',
 'Ċ',
 'ĠĠĠ',
 'Ġdef',
 'Ġgcd',
 '(a',
 ',',
 'Ġb',
 '):Ċ',
 'ĠĠĠĠĠĠĠ',
 'Ġwhile',
 'Ġb',
 ':Ċ',
 'ĠĠĠĠĠĠĠĠĠĠĠ',
 'Ġa',
 ',',
 'Ġb',
 'Ġ=',
 'Ġb',
 ',',
 'Ġa',
 'Ġ%',
 'Ġb',
 'Ċ',
 'ĠĠĠĠĠĠĠ',
 'Ġreturn',
 'Ġa',
 'ĊĊ',
 'ĠĠĠ',
 'Ġdef',
 'Ġlcm',
 '(a',
 ',',
 'Ġb',
 '):Ċ',
 'ĠĠĠĠĠĠĠ',
 'Ġreturn',
 'Ġa',
 'Ġ*',
 'Ġb',
 'Ġ//',
 'Ġgcd',
 '(a',
 ',',
 'Ġb',
 ')ĊĊ',
 'ĠĠĠ',
 'Ġleft',
 ',',
 'Ġright',
 'Ġ=',
 'Ġmin',
 '(A',
 ',',
 'ĠB',
 '),',
 'ĠN',
 'Ġ*',
 'Ġmin',
 '(A',
 ',',
 'ĠB',
 ')Ċ',
 'ĠĠĠ',
 'Ġwhile',
 'Ġleft',
 'Ġ<',
 'Ġright',
 ':Ċ',
 'ĠĠĠĠĠĠĠ',
 'Ġmid',
 'Ġ=',
 'Ġ(',
 'left',
 'Ġ+',
 'Ġright',
 ')',
 'Ġ//',
 'Ġ',
 '2',
 'Ċ',
 'ĠĠĠĠĠĠĠ',
 'Ġif',
 'Ġmid',
 'Ġ//',
 'ĠA',
 'Ġ+',
 'Ġmid',
 'Ġ//',
 'ĠB',
 'Ġ-',
 'Ġmid',
 'Ġ//',
 'Ġlcm',
 '(A',
 ',',
 'ĠB',
 ')',
 

In [51]:
print(sample["generator"])

meta-llama/Llama-3.3-70B-Instruct


In [13]:
Counter(D_ev["language"])

Counter({'Python': 91461, 'C++': 4679, 'Java': 3860})

In [14]:
Counter(D_te["language"])

Counter({'Python': 303,
         'Java': 256,
         'C#': 122,
         'JavaScript': 85,
         'C++': 75,
         'Go': 60,
         'C': 51,
         'PHP': 48})

In [15]:
D_te

,code,generator,label,language
0,public Vector To(Vector o)\n {\n ...,Human,0,C#
1,func (v *DefaultMessageSyntaxValidator) Valida...,Human,0,Go
2,"""""""Module managing testsuite capabilities\n\nC...",Human,0,Python
3,void Anvil::Image::on_memory_backing_opaque_up...,Human,0,C++
4,bool NOMAD::Priority_Eval_Point::dominates\n( ...,Human,0,C++
...,...,...,...,...
995,/**-------------------------------------------...,Human,0,C
996,"void AuxFunc(const dTensor1& xpts, \n\t dT...",Human,0,C++
997,def plot_confusion_matrix(\n context: MLCli...,Human,0,Python
998,import sys\nfrom collections import Counter\n\...,human,0,Python
